<a href="https://colab.research.google.com/github/LC1332/Chat-Haruhi-Suzumiya/blob/main/notebook/%E5%AF%B9%E5%A3%B0%E7%BA%B9%E7%9A%84%E4%BA%A4%E5%8F%89%E6%A3%80%E9%AA%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 对凉宫春日片段声纹识别的交叉验证

**Chat凉宫春日**是模仿凉宫春日等一系列动漫人物，使用近似语气、个性和剧情聊天的语言模型，

<details>
  <summary> 本项目由李鲁鲁，冷子昂，闫晨曦，封小洋，scixing，贾曜恺等开发。 </summary>

李鲁鲁发起了项目，并完成了最早的版本，在多个微信群实现了测试。完成了GPT自动生成对话部分。

冷子昂参与了早期Gradio的开发，并且参与了后端和前端的选型。debug和最终上线了app.py

闫晨曦将李鲁鲁的notebook重构为app.py

封小洋进行了中文转日文模型的选型，完成了针对台词抽取图片的工具，（将要完成）和haruhi图片分类器

scixing实践了VITS语音，完成了台词对应的语音抽取，（将要完成）特定人物的语音分类器。

贾曜恺开发了一个前端实现方案，待测试。

</details>

本脚本由李鲁鲁开发，用于验证声纹特征交叉验证的准确率

- [x] 整理之前的GPTData文件夹
- [x] 确认fxy和scixing是否做了这个事情
- [x] 转移声纹和声音文件夹到GoogleDrive
- [x] 解析标注文件夹，将特征整理成feature，label形式
- [x] 进行交叉验证
- [x] 给出自动阈值
- [x] 进行分类尝试

In [1]:
#mount google drive to colab
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
!ls /content/drive/MyDrive/GPTData/HaruhiRoleVoices

'Haruhi ep10.zip'  'Haruhi ep1.zip'   voiceWithCha.zip


In [26]:
#copy all zip to /content
!cp /content/drive/MyDrive/GPTData/HaruhiRoleVoices/*.zip /content/



In [ ]:
#unzip all zip file
!unzip -q "/content/Haruhi ep10.zip"
!unzip -q "/content/Haruhi ep1.zip"
# !unzip "/content/voiceWithCha.zip"

In [6]:
#unzip /content/voiceWithCha.zip into /content/roles quitely
!unzip -q /content/voiceWithCha.zip -d /content/roles

In [16]:
# scan subfolder in /content/roles and save into a list
import os
roles = os.listdir('/content/roles')

# for each role in roles, scan /content/roles/[role] and save into a list of list
roles_list = []
roles_list_full = []
for role in roles:

    sub_list = os.listdir(os.path.join('/content/roles',role))
    roles_list.append(sub_list)

    full_name_list = ['/content/roles/'+role+'/'+file for file in sub_list]
    roles_list_full.append(full_name_list)


In [17]:
print(roles_list[0])

['68_而是依照凉宫同学的思维模式进行思考而得出的结论.wav', '55_就需要到和那些事件相应的场所去才行.wav', '51_但是这里说那些现实主义的话就太煞风景了.wav', '63_换言之  名侦探所到之处必有事件发生.wav', '54_想要遭遇这种非现实的神秘事件.wav', '44_推理故事作品里的名侦探们.wav', '58_的确  现实世界是不会像故事那样的.wav', '31_我有个非常有钱的远房亲戚.wav', '32_他的钱多到可以买下一个无人岛并且在上面盖上别墅.wav', '50_这样的事件只存在于虚构而非现实的故事世界中.wav', '43_那么你想想「名侦探」那三个字吧.wav', '56_因为创作出来的名侦探们都是这样被卷进事件中的.wav', '349_新川  有什么事吗？.wav', '59_但是现在我们所要去的「孤岛」这种地方.wav', '60_一般人都认为是最适合成为杀人事件舞台的地方.wav', '193_是除我们之外应邀前来的客人.wav', '69_也就是说呢  她想尝试一次看看.wav', '37_臣拜领此职.wav', '52_因为凉宫同学似乎就是想要投身于这样的虚构世界中.wav', '67_「名侦探」和「Closed Circle」并不是我所空想出来的.wav', '64_被称为名侦探的人都有能够呼唤事件的超自然能力.wav', '346_怪不得凉宫同学吃过早饭后.wav', '45_不知道为什么会不停地卷入难以理解的事件中.wav', '33_那栋别墅前几天刚刚落成.wav', '66_我有时候也会偶尔认真一回的.wav']


In [21]:
candidate_path = [
    "/content/Haruhi ep1",
    "/content/Haruhi ep10"
]

In [45]:
import pickle
import numpy as np


features = []
labels = []
dim = 0

count = 0

for role in roles:
    print(role,end=' ')

    for file in roles_list[roles.index(role)]:
        # print(file)

        deal_flag = False

        for candidate in candidate_path:

            candidate_fname = os.path.join(candidate,'voice',file)

            if os.path.exists(candidate_fname):
                # print(candidate_fname,'found')
                deal_flag = True
                feature_fname = os.path.join(candidate,'feature',file) + '.pkl'
                break

        if deal_flag == False:
            print('warning!',file,'not found')
            continue

        if not os.path.exists(feature_fname):
            print('warning!',feature_fname,'not found')
            continue

        # pinkle load feature_fname
        with open(feature_fname,'rb') as f:
            feature = pickle.load(f)

        count += 1

        # append numpy array feature into numpy matrix features
        if dim == 0:
            features = feature
            dim = feature.shape[0]
            # print(dim)
        else:
            features = np.vstack((features,feature))

        labels.append(role)

        # print(feature_fname,'found')



    # break


古泉 多丸裕 谷口 新川先生 阿虚妹妹 长门 春日 朝仓 阿虚 森小姐 忘了叫啥的角色 多丸圭一 朝比奈 

feature是一个N*D的numpy矩阵，每行存储了一个D维特征
labels是一个python的list of string，表示每行对应的数据的标签。
我想验证这批数据使用K近邻分类，在10折交叉时的准确率，请用python为我实现。

In [50]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

# 创建 K 近邻分类器模型
k = 1  # 假设选择 k 值为 3
knn = KNeighborsClassifier(n_neighbors=k, metric='cosine')

# 将特征转换为 NumPy 数组
features = np.array(features)

# 将标签转换为 NumPy 数组
labels = np.array(labels)

# 计算 10 折交叉验证的准确率
cv_accuracy = cross_val_score(knn, features, labels, cv=5)

# 打印每折交叉验证的准确率
for fold, accuracy in enumerate(cv_accuracy, 1):
    print(f"Fold {fold}: {accuracy}")

# 打印平均准确率
mean_accuracy = np.mean(cv_accuracy)
print(f"Average Accuracy: {mean_accuracy}")

Fold 1: 0.8809523809523809
Fold 2: 0.9047619047619048
Fold 3: 0.8780487804878049
Fold 4: 0.8536585365853658
Fold 5: 0.8536585365853658
Average Accuracy: 0.8742160278745643


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


定义一个函数，给定roles和roles_list，输出feature和label

In [51]:
#@title 定义gather_feature_label

def gather_feature_label( roles, roles_list ):
    features = []
    labels = []
    dim = 0

    count = 0

    for role in roles:
        print(role,end=' ')

        for file in roles_list[roles.index(role)]:
            # print(file)

            deal_flag = False

            for candidate in candidate_path:

                candidate_fname = os.path.join(candidate,'voice',file)

                if os.path.exists(candidate_fname):
                    # print(candidate_fname,'found')
                    deal_flag = True
                    feature_fname = os.path.join(candidate,'feature',file) + '.pkl'
                    break

            if deal_flag == False:
                print('warning!',file,'not found')
                continue

            if not os.path.exists(feature_fname):
                print('warning!',feature_fname,'not found')
                continue

            # pinkle load feature_fname
            with open(feature_fname,'rb') as f:
                feature = pickle.load(f)

            count += 1

            # append numpy array feature into numpy matrix features
            if dim == 0:
                features = feature
                dim = feature.shape[0]
                # print(dim)
            else:
                features = np.vstack((features,feature))

            labels.append(role)

    return features, labels




In [95]:
import random

roles_sel = []
roles_list_sel = []

M = 8

for role in roles:
    wav_list = roles_list[roles.index(role)]

    # if len(wav_list) < M:
        # continue

    # random pick 5 element from wav_list
    random.shuffle(wav_list)
    # wav_list = wav_list[:]

    roles_sel.append(role)
    roles_list_sel.append(wav_list)

In [96]:
feat_sel, label_sel = gather_feature_label(roles_sel,roles_list_sel)


古泉 多丸裕 谷口 新川先生 阿虚妹妹 长门 春日 朝仓 阿虚 森小姐 忘了叫啥的角色 多丸圭一 朝比奈 

feature是一个N*D的numpy矩阵，每行存储了一个D维特征
labels是一个python的list of string，表示每行对应的数据的标签。

实现一个Python类My_Classifier,
这个类可以使用feature和labels初始化一个 sklearn的1-NN的最近邻分类器，使用cosine度量
这个类有一个predict方法，输出一个tuple，为(类别，距离)

In [97]:
import numpy as np
from scipy.spatial.distance import cosine

class My_Classifier:
    def __init__(self, feature, labels):
        self.feature = feature
        self.labels = labels

    def predict(self, x):
        min_dist = float('inf')
        predicted_label = None

        for i, f in enumerate(self.feature):
            dist = cosine(x, f)
            if dist < min_dist:
                min_dist = dist
                predicted_label = self.labels[i]

        return predicted_label, min_dist

In [98]:
my_classifier = My_Classifier(feat_sel, label_sel)

corrent_dists = []
wrong_dists = []

for i in range(len(labels)):
    # read i-th row from features, save as feat
    feat = features[i,:]
    # read i-th row from labels, save as label
    label = labels[i]

    # predict label of i-th row
    predicted_label, distance = my_classifier.predict(feat)

    # if distance < 1e-3:
    #     continue

    if label == predicted_label:
        corrent_dists.append(distance)
    else:
        wrong_dists.append(distance)


corrent_dists 和 wrong_dists 都是python的list of float, 统计两者的均值和方差，用plotly将两个高斯分布进行可视化，使用不同的颜色标示两个分布。

In [72]:
#@title 高斯可视化

import numpy as np
import plotly.graph_objects as go

# 计算均值和方差
corrent_mean = np.mean(corrent_dists)
corrent_var = np.var(corrent_dists)
wrong_mean = np.mean(wrong_dists)
wrong_var = np.var(wrong_dists)

# 可视化高斯分布
x = np.linspace(min(corrent_dists + wrong_dists), max(corrent_dists + wrong_dists), 100)
corrent_distribution = (1 / np.sqrt(2 * np.pi * corrent_var)) * np.exp(-0.5 * ((x - corrent_mean) / np.sqrt(corrent_var)) ** 2)
wrong_distribution = (1 / np.sqrt(2 * np.pi * wrong_var)) * np.exp(-0.5 * ((x - wrong_mean) / np.sqrt(wrong_var)) ** 2)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=corrent_distribution, mode='lines', name='Correct Distribution'))
fig.add_trace(go.Scatter(x=x, y=wrong_distribution, mode='lines', name='Wrong Distribution'))

# 添加布局和标题
fig.update_layout(
    title='Gaussian Distributions',
    xaxis_title='Distance',
    yaxis_title='Probability Density',
)

# 显示图表
fig.show()


corrent_dists 和 wrong_dists 都是python的list of float, 统计两者的均值和方差。

然后计算 P( correct | dist )函数，即

P( correct | dist ) = P( dist | corret ) / (P( dist | corret ) + P( dist | wrong ) )

用plotly进行可视化

In [73]:
import numpy as np

corrent_mean = np.mean(corrent_dists)
corrent_var = np.var(corrent_dists)
wrong_mean = np.mean(wrong_dists)
wrong_var = np.var(wrong_dists)

def gaussian_pdf(x, mean, var):
    return (1 / np.sqrt(2 * np.pi * var)) * np.exp(-0.5 * ((x - mean) / np.sqrt(var)) ** 2)

x = np.linspace(min(corrent_dists + wrong_dists), max(corrent_dists + wrong_dists), 100)
p_dist_corrent = gaussian_pdf(x, corrent_mean, corrent_var)
p_dist_wrong = gaussian_pdf(x, wrong_mean, wrong_var)

import plotly.graph_objects as go

p_correct_given_dist = p_dist_corrent / (p_dist_corrent + p_dist_wrong)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=p_correct_given_dist, mode='lines', name='P(corrent | dist)'))

# 添加布局和标题
fig.update_layout(
    title='P(Corrent | Dist)',
    xaxis_title='Distance',
    yaxis_title='Probability',
)

# 显示图表
fig.show()

In [99]:
threshold_certain = 0.4
threshold_doubt = 0.6

In [104]:
feature_folder =  "/content/Haruhi ep1/feature/"
save_name = "/content/output_ep1.txt"

In [106]:
feature_folder =  "/content/Haruhi ep10/feature/"
save_name = "/content/output_ep10.txt"

In [107]:
# count how many files exist in feature_folder
import os
N_files = len( os.listdir(feature_folder) )
N_files += 100

feature_folder =  "/content/Haruhi ep1/feature/"

在feature_folder下，有形如 <id>_<台词>.wav.pkl的文件
如:
0_要到几岁才开始不相信圣诞老人的存在.wav.pkl

将id从0到N_files循环，

在feature_folder下寻找以id开头,'.wav.pkl'结尾的文件

并且提取id的字符串打印

使用python实现

In [108]:
import os

file_list = os.listdir(feature_folder)



with open(save_name, "w", encoding="utf-8") as f_out:

    for id in range(N_files):

        deal_file_name = '';

        for file in file_list:
            if file.startswith(str(id) + '_') and file.endswith('.wav.pkl'):
                deal_file_name = file
                id_str = file.split('_')[1]
                break

        if deal_file_name == '':
            # print('file not found')
            continue

        full_file_name = os.path.join(feature_folder, deal_file_name)

        with open(  full_file_name,'rb') as f:
            feature = pickle.load(f)

        predicted_label, distance = my_classifier.predict(feature)

        role_name = ''

        if distance < threshold_certain:
            role_name = predicted_label
        elif distance < threshold_doubt:
            role_name = '(可能)' + predicted_label

        output_str = role_name + ':「' +  id_str[:-8] + '」'

        # print(output_str)
        f_out.write(output_str + "\n")

    # break
